In [1]:
%matplotlib widget

In [2]:

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import scipy as sp
import databento as db 
import zstandard as zstd
from databento import DBNStore
import math
import matplotlib.animation as animation # for animation

In [4]:
# import polars

import polars as pl

In [5]:
bclr=['r','b','g','m','y','c','k']

notebook_dir="/home/rupam/Rupam_pcloud/Quantitative Finance/Quant_researcher_Job/Strategic_prep/My_projects/"
data_path=notebook_dir+"Mini_S_and_P_futures_data/LOB_10/"


In [6]:
files=[data_path+f'glbx-mdp3-{day}.mbp-10.parquet' for day in ['20220202','20220203','20220204','20220211']]

In [ ]:
%%time
df = pl.read_parquet(files)

In [ ]:
df

In [ ]:
df.columns;
lvl_lst=['00']#,'02','04','06']
cols=['ts_event','action','side','price','size']+[col for col in df.columns if any(lvl in col for lvl in lvl_lst)]+['symbol']
cols;

In [ ]:
df['action']

In [ ]:
df_ESH2.col('ts_event')

In [ ]:
df_ESH2['ts_event']=df_ESH2.col('ts_event').str.strptime(pl.Datetime, strict=False)

In [ ]:
df=df[cols].sort('ts_event')
df_ESH2=df.filter((df['price']!=9223372036854775807) & (df['symbol']=='ESH2') & (df['bid_px_00'] < df['ask_px_00']))
df_ESH2['ts_event']=df_ESH2.col("ts_event").str.strptime(pl.Datetime, strict=False)

#df_ESH2=df_ESH2['2022-02-02 09:30:00':'2022-02-02 16:00:00']
df_ESH2

In [ ]:
df_ESH2.head(5)

In [ ]:
df_ESH2.iloc[2]

In [ ]:
class BasicProp:
    def __init__(self,df):
        self.df=df
        self.lvl_list=lvl_lst
        
    def mid_price(self):
        mids={}
        for lvl in self.lvl_list:
            mids[f'mid_px_{lvl}']=(self.df[f'ask_px_{lvl}']+self.df[f'bid_px_{lvl}'])/2
        return pd.DataFrame(mids, index=self.df.index)
        
    def spread(self):
        sprd={}
        for lvl in self.lvl_list:
            sprd[f'spread_{lvl}']=self.df[f'ask_px_{lvl}']-self.df[f'bid_px_{lvl}']
        return pd.DataFrame(sprd, index=self.df.index)

    def rel_spread(self):
        rel_sprd={}
        for lvl in self.lvl_list:
            rel_sprd[f'rel_spread_{lvl}']=2*(self.df[f'ask_px_{lvl}']-self.df[f'bid_px_{lvl}'])/(self.df[f'ask_px_{lvl}']+self.df[f'bid_px_{lvl}'])
        return pd.DataFrame(rel_sprd, index=self.df.index)

    def ob_imbl(self):
        imbl={}
        for lvl in self.lvl_list:
            imbl[f'ob_imbl_{lvl}']=(self.df[f'bid_sz_{lvl}']-self.df[f'ask_sz_{lvl}'])/(self.df[f'bid_sz_{lvl}']+self.df[f'ask_sz_{lvl}'])
        return pd.DataFrame(imbl, index=self.df.index)

    def quote_return(self,case='log'):
        rtn={}
        for lvl in self.lvl_list:
            mid=(self.df[f'ask_px_{lvl}']+self.df[f'bid_px_{lvl}'])/2
            if case =='abs':
                rtn[f'abs_return_{lvl}']=mid.diff()/mid.shift(1) # or mid.pct_change()
            elif case =='log':
                rtn[f'log_return_{lvl}']=np.log(mid).diff()
        return pd.DataFrame(rtn, index=self.df.index)

    def volatility(self,win=1000):
        vol={}
        for lvl in self.lvl_list:
            mid=(self.df[f'ask_px_{lvl}']+self.df[f'bid_px_{lvl}'])/2
            rtn=mid.diff()/mid.shift(1)
            vol[f'vol_{lvl}_{win}']=rtn.rolling(window=win).std()
        return pd.DataFrame(vol, index=self.df.index)


In [ ]:
ESH2_prop=BasicProp(df_ESH2)

In [ ]:
def plot_time_series(df, title='Order Book Features', steps=10,):
    cols=df.columns
    n = len(cols)
    ncols= 2 if n>1 else 1
    nrows= math.ceil(n / ncols)
     

    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(5.5 * ncols, 3.5 * nrows))
    axs = axs.flatten() if n > 1 else [axs]

    lines = []
    y_list = []

    for i in range(n):
        data = df.iloc[::steps]
        x = data.index.values
        y = data.values[:,i]
        ax = axs[i]
        ax.set_xlim(np.nanmin(x), np.nanmax(x))
        ax.set_ylim(np.nanmin(y), np.nanmax(y))
        ax.set_xlabel("Time index (sampled)")
        ax.set_ylabel("Value")
        ax.grid(True)
        line, = ax.plot([], [], label=cols[i], color=bclr[i], alpha=0.5)
        ax.legend()
        lines.append(line)
        y_list.append(y)    

    for j in range(i + 1, len(axs)):
        fig.delaxes(axs[j])

    def update(frame):
        for i in range(n):
            lines[i].set_data(x[:frame], y_list[i][:frame])
        return lines

    ani = animation.FuncAnimation(fig, update, frames=len(x), interval=20, blit=False, repeat=False)

    fig.suptitle(title, fontsize=16)
    fig.tight_layout()
    plt.show()

    return ani


In [ ]:
df_combined = pd.concat([ESH2_prop.spread(), ESH2_prop.mid_price(), ESH2_prop.ob_imbl(),ESH2_prop.quote_return(),ESH2_prop.volatility()], axis=1)

In [ ]:
plot_time_series(df_combined,steps=10000)

In [ ]:
#resampling in 1s and 5s bars

def time_resampler(df,delt):
    t0=pd.Timestamp('2022-02-02 09:30:00')
    dict_rs={}
    dt=pd.Timedelta(seconds=delt)
    while t0<df.index[-1]-dt:
        df_dt=df[t0:t0+dt]
        prop=BasicProp(df_dt)
        avg_pr=prop.mid_price().mean().values[0]
        avg_spread=prop.spread().mean().values[0]
        avg_return=prop.quote_return().mean().values[0]
        avg_imbl=prop.ob_imbl().mean().values[0]
        dict_rs[t0+dt]=(avg_pr,avg_spread,avg_return,avg_imbl)
        t0=t0+dt
    df_rs=pd.DataFrame.from_dict(dict_rs,orient='index',columns=['avg_mid_price','avg_spread','avg_return','avg_imbl'])
    df_rs.index.name='ts_event'
    return df_rs

In [ ]:
df1s_ESH2=time_resampler(df_ESH2,1)

In [ ]:
df1s_ESH2

In [ ]:
plot_time_series(df1s_ESH2,'Average Feateures over 1s window', steps = 100)

In [ ]:
df5s_ESH2=time_resampler(df_ESH2,5)

In [ ]:
plot_time_series(df5s_ESH2,'Average Feateures over 5s window', steps = 20)

In [ ]:
def plot_histogram(df,title='Histogram of avg quantities',bins=100):
    cols=df.columns
    n = len(cols)
    ncols= 2 if n>1 else 1
    nrows= math.ceil(n / ncols)
     

    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(5.5 * ncols, 3.5 * nrows))
    axs = axs.flatten() if n > 1 else [axs]


    for i in range(n):
        ax = axs[i]
        ax.set_xlabel(cols[i])
        ax.set_ylabel("Frequency")
        ax.grid(True)
        counts,bin_edges,patches = ax.hist(df[cols[i]],bins=bins, label=cols[i], color=bclr[i],edgecolor='black',alpha=0.5)
        ax.set_xlim(np.nanmin(bin_edges), np.nanmax(bin_edges))
        #ax.set_ylim(np.nanmin(counts), np.nanmax(counts))
        ax.legend()

    for j in range(i + 1, len(axs)):
        fig.delaxes(axs[j])

    fig.suptitle(title, fontsize=16)
    fig.tight_layout()
    plt.show()

In [ ]:
plot_histogram(df1s_ESH2,bins=100)


In [ ]:
plot_histogram(df5s_ESH2,bins=100)

In [ ]:
fig=plt.figure(figsize=(6,4))
plt.scatter(df_combined['log_return_00'][::10],df_combined['ob_imbl_00'][::10],color=bclr[0],alpha=0.5)
plt.xlabel('OB imbalance')
plt.ylabel('Fwd price change')
plt.show()

In [ ]:
df_combined['ob_imbl_00'].corr(df_combined['log_return_00'])

In [ ]:
plot_histogram(df_combined,'Histogram of Order book',bins=100)